# <font color="blue">Fouille de données textuelles : projet de fin de cours</font>

*M2 MAS parcours Science des données 2019-2020, novembre 2019*

*BERARD Gwenaël - BIC Maxime - CHAUVIN Antoine - LANCELLE Etienne*

### <font color="green">Objectif</font>

Création d'un agent de dialogue (informellement et souvent abusivement appelé *chatbot*) :
+ en français. Des données conversationnelles dans des langues autres que l'anglais étant rares et pas faciles à obtenir, vous constituerez vos propres données. Le volume sera petit et votre chatbot sera donc très limité dans ses "capacités" ;
+ hybride : principalement orienté vers la résolution d'une tâche (répondre à une question métier), mais comme solution de repli purement conversationnel (*chatbot* : conversation sans finalité pratique) ;
+ approche la plus simple possible : chaque réponse de l'agent dépend uniquement du tour de parole précédent de l'usager. La conversation ne s'arrête pas, mais il n'y a pas de continuité dans le dialogue.

## **<font color="red">Composantes et fonctionnement</font>**

### <font color="red">1. Composante orientée tâche</font>

Approche basée sur un corpus : l'agent retourne une réponse toute faite extraite d'un corpus préexistant (*retrieval-based* agent).

#### Constitution du corpus

Par Web scraping : à partir d'un site Web qui contient une section FAQ, récupérez des paires question-réponse. Plus il y aura des questions, plus ce sera intéressant.

Quelques exemples :
+ https://particulier.edf.fr/fr/accueil/aide-et-contact/aide/faq/questions-les-plus-frequentes.html (attention, il y a plusieurs pages, cliquer sur les différents onglets pour voir toutes les questions)
+ https://fr.wikipedia.org/wiki/Aide:FAQ

Il faut donc constituer une base de questions, chacune avec la réponse associée. Si les données s'y prêtent (questions sur des domaines bien distincts, comme [ici](https://fr.wikipedia.org/wiki/Aide:FAQ), et en nombre assez important), les questions-réponses peuvent en plus être regroupées par catégories thématiques.

In [1]:
from bs4 import BeautifulSoup
import requests

liste_id = ["194","196","478","487","575"]
liste_scrap = [] 
for i in liste_id:
  response = requests.get("http://www.chateauversailles.fr/preparer-ma-visite/faq" + "?tid=" + i)
  soup = BeautifulSoup(response.text, "html.parser")
  faq = soup.find("div", class_ = "view-content").get_text()
  for i in faq.split("\n"):
    if i != '':
      liste_scrap.append(i)

Ici on importe BeautifoulSoup, nécessaire au web-scrapping. Nous avons vu sur le site que les 5 onglets de la FAQ étaient liés à des id. Nous récupérons ces id et faisons ensuite une boucle sur tous ces onglets, afin de récupérer les questions. Nous disposons alors d'une longue liste de 94 éléments, où questions et réponses sont alternées.

In [2]:
liste_question = []
liste_reponse = []

for i,e in enumerate(liste_scrap):
  if i % 2 == 0:
    liste_question.append(e)
  elif i % 2 != 0:
    liste_reponse.append(e)

In [3]:
import pandas as pd
dico = {
    "Question" : liste_question,
    "Reponse" : liste_reponse
}
data = pd.DataFrame(dico)
data.head()

,Question,Reponse
0,Où acheter les billets ? Est-il nécessaire de ...,"Réservation en ligne :Pour gagner du temps, ac..."
1,Où dois-je me présenter avec mes billets achet...,Visite libre du Château :Si vous avez réservé ...
2,Quels sont les avantages et les tarifs des vis...,"Lors d'une visite guidée, un conférencier du c..."
3,Quels sont les moyens de paiement acceptés ?,Les moyens de paiement acceptés en caisse sont...
4,Existe-t-il un billet pour l’ensemble du Domai...,Le billet Passeport permet d’accéder à l’ensem...


On répartit les questions et les réponses dans les listes correspondantes, puis on rassemble la FAQ dans un DataFrame.

#### Classification de la question

Lorsque l'agent reçoit une question, il devra décider si la question est réellement liée au domaine métier ou non. Si oui et si les données sont en plus regroupées par thématiques, une deuxième décision est à prendre : sur laquelle de ces thématiques porte la question.

Si c'est une question métier, le chatbot retournera une réponse pertinente selon sa stratégie ; si non, il déclenchera la composante conversationnelle, qui produira une réponse originale.

Il faut donc mettre en place une stratégie pour la prise de ces décisions et pour la sélection de la réponse.

Quelle que soit l'approche il faudra d'abord :
+ prétraiter la base de données (à faire une seule fois et à stocker). Attention, si on vectorise le corpus il faudra garder le vectoriseur (l'enregistrer comme `pickle`) pour appliquer ensuite le même vectoriseur à la question ;
+ prétraiter la question (en temps réel).

In [4]:
import spacy
import nltk
nlp = spacy.load('fr_core_news_sm')

Afin de mesurer la similarité entre la nouvelle question entrée par l'utilisateur et les questions de la base, nous allons appliquer différents traitements aux textes. Certains méthodes sont efficaces, d'autres le sont moins, nous l'allons montrer tout à l'heure.

In [5]:
def lemmatise_text(text):
    tokeni = nlp(text)
    lem = []
    for i in tokeni:
        lem.append(i.lemma_)
    resu = " ".join(lem)
    resu = nlp(resu)
    return resu

On crée la fonction permettant de lemmatiser les questions/réponses.

In [6]:
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.stem import SnowballStemmer # ou: from nltk.stem.snowball import FrenchStemmer
def stem_text(text):
    stemmer = SnowballStemmer('french')
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    rac = []
    for i in tokenizer.tokenize(text):
        rac.append(stemmer.stem(i))
    return(nlp(" ".join(rac)))

On crée la fontion permettant de créer des Stems à partir des questions/réponses.

In [7]:
def replace_words_with_pos_tag(text):
    txt_nlp = nlp(text)
    liste_pos = []
    for token in txt_nlp:
        liste_pos.append(token.pos_)
    return(nlp(" ".join(liste_pos)))

On crée la fontion permettant de retrouver les catégories des mots contenus dans les questions/réponses.

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\berar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir', 'al']
def virer_sw(text):
    txt_nlp = nlp(text)
    liste_mot = []
    for token in txt_nlp:
        if str(token) not in sw:
            liste_mot.append(str(token))
    return(nlp(" ".join(liste_mot)))

On crée la fonction permettant de retirer les mots vides des questions/réponses.

In [10]:
traitement_question = pd.DataFrame({
    "Question" : data.Question,
    "Question_token" : [nlp(x) for x in data.Question],
    "Question_lemme" : data.Question.apply(lemmatise_text),
    "Question_stem" : data.Question.apply(stem_text),
    "Question_pos" : data.Question.apply(replace_words_with_pos_tag),
    "Question_sw" : data.Question.apply(virer_sw)
})

traitement_reponse = pd.DataFrame({
    "Reponse" : data.Reponse,
    "Reponse_token" : [nlp(x) for x in data.Reponse],
    "Reponse_lemme" : data.Reponse.apply(lemmatise_text),
    "Reponse_stem" : data.Reponse.apply(stem_text),
    "Reponse_pos" : data.Reponse.apply(replace_words_with_pos_tag),
    "Reponse_sw" : data.Reponse.apply(virer_sw)
})
traitement_question

,Question,Question_token,Question_lemme,Question_stem,Question_pos,Question_sw
0,Où acheter les billets ? Est-il nécessaire de ...,"(Où, acheter, les, billets, ?, Est, -, il, néc...","(où, acheter, le, billet, ?, être, -, il, néce...","(où, achet, le, billet, ?, est-il, nécessair, ...","(ADV, VERB, DET, NOUN, PUNCT, VERB, PUNCT, PRO...","(Où, acheter, billets, ?, Est, -, nécessaire, ..."
1,Où dois-je me présenter avec mes billets achet...,"(Où, dois, -, je, me, présenter, avec, mes, bi...","(où, devoir, -, je, me, présenter, avec, mon, ...","(où, dois, -, j, me, présent, avec, me, billet...","(ADV, AUX, PUNCT, PRON, PRON, VERB, ADP, DET, ...","(Où, dois, -, présenter, billets, achetés, pré..."
2,Quels sont les avantages et les tarifs des vis...,"(Quels, sont, les, avantages, et, les, tarifs,...","(quel, être, le, avantage, et, le, tarif, un, ...","(quel, sont, le, avantag, et, le, tarif, de, v...","(ADJ, AUX, DET, NOUN, CCONJ, DET, NOUN, DET, N...","(Quels, avantages, tarifs, visites, guidées, ?)"
3,Quels sont les moyens de paiement acceptés ?,"(Quels, sont, les, moyens, de, paiement, accep...","(quel, être, le, moyen, de, paiement, accepter...","(quel, sont, le, moyen, de, pai, accept, ?)","(ADJ, AUX, DET, NOUN, ADP, NOUN, VERB, PUNCT)","(Quels, moyens, paiement, acceptés, ?)"
4,Existe-t-il un billet pour l’ensemble du Domai...,"(Existe, -, t, -, il, un, billet, pour, l’, en...","(existe, -, t, -, il, un, billet, pour, l, ’, ...","(existe, -, t, -, il, un, billet, pour, l, ’, ...","(NOUN, PUNCT, NOUN, PUNCT, PRON, DET, NOUN, AD...","(Existe, -, -, billet, l, ’, ensemble, Domaine..."
5,Combien coûte un billet pour le château de Ver...,"(Combien, coûte, un, billet, pour, le, château...","(combien, coûte, un, billet, pour, le, château...","(combien, coût, un, billet, pour, le, château,...","(ADV, VERB, DET, NOUN, ADP, DET, NOUN, ADP, NO...","(Combien, coûte, billet, château, Versailles, ..."
6,Comment éviter les files d'attente aux caisses ?,"(Comment, éviter, les, files, d', attente, aux...","(comment, éviter, le, file, de, attente, al, c...","(comment, évit, le, fil, d', attent, aux, cais...","(ADV, VERB, DET, NOUN, ADP, NOUN, ADJ, NOUN, P...","(Comment, éviter, files, d, ', attente, caisse..."
7,Ai-je besoin d'un billet si je bénéficie de la...,"(Ai, -, je, besoin, d', un, billet, si, je, bé...","(ai, -, je, besoin, de, un, billet, si, je, bé...","(ai, -, j, besoin, d', un, billet, si, je, bén...","(NOUN, PUNCT, PRON, NOUN, ADP, DET, NOUN, SCON...","(Ai, -, besoin, d, ', billet, si, bénéficie, g..."
8,Les jardins sont-ils payants les jours de Gran...,"(Les, jardins, sont, -, ils, payants, les, jou...","(le, jardin, être, -, il, payant, le, jour, de...","(le, jardin, sont, -, il, pai, le, jour, de, g...","(DET, NOUN, AUX, PUNCT, PRON, NOUN, DET, NOUN,...","(Les, jardins, -, payants, jours, Grandes, Eau..."
9,Puis-je venir avec des bagages volumineux au C...,"(Puis, -, je, venir, avec, des, bagages, volum...","(puis, -, je, venir, avec, un, bagage, volumin...","(puis, -, j, ven, avec, de, bagag, volumin, au...","(CCONJ, PUNCT, PRON, VERB, ADP, DET, NOUN, ADJ...","(Puis, -, venir, bagages, volumineux, Château, ?)"


On observe ici l'ensemble des questions/réponses, dans leur état original puis dans leurs formes modifiées par les fonctions sus-créées.

In [11]:
traitement_question.to_pickle('trait_question.pkl')
traitement_reponse.to_pickle('trait_reponse.pkl')

On enregistre ces tableaux dans un pickle pour pouvoir facilement les reprendre.

Une fois cela fait, une approche possible est de comparer la question avec chaque question de la base de données, par exemple :
+ en calculant une mesure de leur similarité (cf. `spacy` : attribut `similarity` disponible sur les tokens mais aussi les documents). Si la similarité est en-dessous d'un certain seuil, on considère que ce n'est pas une question métier (à noter que cela n'est pas nécessairement vrai, mais cela simplifie les choses) et on bascule sur la composante générative du chatbot ;
+ en cherchant dans la question reçue des mots-clés associés à chaque question de la base de données. On peut là encore se servir de similarités entre vecteurs de mots pour identifider des mots similaires aux mots-clés prédéfinis. On peut aussi tirer parti de l'étiqueteur pré-entraîné d'entités nommées dans un modèle statistique de `spacy`. Même hypothèse : en cas d'échec, on considère que la question est hors sujet (ou du moins hors domaine) et on déclenche la composante générative du chatbot.

Cette approche devient moins pratique si la base de données est grande. Dans ce cas il devient plus intéressant d'organiser la base de données en sections de moindre taille.

À noter que, au lieu de comparer la nouvelle question avec les questions de la base, on peut aussi comparer la nouvelle question avec les réponses de la base. Cela peut sembler surprenant, mais des travaux publiés ont montré que cette approche est parfois plus efficace.

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
def score_texte(question):
#    score_token_q = []
#    score_lemme_q = []
    score_stem_q = []
#    score_pos_q = []
#     score_sw_q = []
    
    for i in range(0,48):
#         score_lemme_q.append(lemmatise_text(question).similarity(traitement_question.Question_lemme[i]))
#         score_token_q.append(nlp(question).similarity(traitement_question.Question_token[i]))
         score_stem_q.append(stem_text(question).similarity(traitement_question.Question_stem[i]))
#         score_pos_q.append(replace_words_with_pos_tag(question).similarity(traitement_question.Question_pos[i]))
#         score_sw_q.append(virer_sw(question).similarity(traitement_question.Question_sw[i]))
        
    dico = {
        "Question" : traitement_question.Question,
#         "QToken" : score_token_q,
#         "QLemme" : score_lemme_q,
         "QStem" : score_stem_q,
#         "QPos" : score_pos_q,
#          "QSw" : score_sw_q
    }
    
    data = pd.DataFrame(dico)
    return(round(data,2))

Ici, on crée la table des similarités, entre la nouvelle question saisie et toutes les questions de la base. Le temps de traitement est grand, nous avons choisi de retirer certains traitement, pour ne garder que le Stem. Nous avons choisi de garder cette méthode car nous trouvions qu'elle discriminait au mieux les nouvelles questions, entre "questions pertinentes" et "questions hors-sujet". <br>
Le tableau ci-dessous nous donne la similarité de la nouvelle question avec toutes les questions de la base.
Nous n'avons pas comparé la nouvelle question aux réponses, car le temps de calcul était trop important et le Chatbot n'était pas du tout dynamique.

In [15]:
import numpy as np
tableau = score_texte("Bagages volumineux ?")
print(np.max(tableau.QStem))
tableau.iloc[np.argmax(tableau.QStem),0]

0.61


'Combien coûte un billet pour le château de Versailles ? Où puis-je l’acheter ?'

Une autre option serait d'apprendre :
+ un classifieur binaire capable de distinguer entre thématiques liées au métier et autres thématiques. Pour cela on peut utiliser pour l'apprentissage les données métier de notre base de questions-réponses et une partie du corpus de la composante conversationnelle (équilibrer les deux corpus en termes de taille) ;
+ si pertinent, un deuxième classifieur, cette fois-ci multi-classe, qui distinguera entre les thématiques métier. Les données d'apprentissage seront les données de notre base de questions-réponses, classées en catégories.

Si on opte pour cette approche, le(s) classifieur(s) devront être appris en amont de la mise en "production" du chatbot et stockés pour une utilisation en temps réel (`pickle`).

### <font color="red">2. Composante conversationnelle</font>

Approche générative : le chatbot produit du texte original en prenant comme point de départ le tour de parole (la question) de l'utilisateur.

#### Constitution du corpus

Nous utiliserons pour cette composante un corpus de sous-titres de films en français. Le projet [OPUS](http://opus.nlpl.eu/) met à disposition un grand nombre de corpus multilingues, dont des sous-titres de films, avec ou sans alignement entre texte source et texte cible.

Pour cette application nous utiliserons exclusivement la partie française de l'un des nombreux jeux de données disponibles [ici](http://opus.nlpl.eu/OpenSubtitles-v2018.php). Pour cela, il faut télécharger l'une des archives au format `.txt`, qui sont disponibles dans le deuxième tableau de la page Web (le tableau en couleurs sous la section Statistics and TMX/Moses Downloads). Choisissez l'une des cellules correspondant au français en-dessous de la diagonale ; optez pour une taille moyenne (les fichiers sont volumineux et ils contiennent beaucoup de texte).

Une fois l'archive téléchargée, décompressez-la et ne retenez que le fichier qui se termine par `-fr.fr`.

#### Apprentissage du modèle

Nous apprendrons un modèle de langue sur ces données. Un tutoriel pour un modèle basé sur les caractères est disponible [ici](https://www.tensorflow.org/tutorials/text/text_generation). La même approche peut s'appliquer au niveau des mots plutôt que des caractères.

Une fois le modèle appris stockez-le comme `pickle`. Lors du fonctionnement en temps réel il prendra en entrée le texte de l'usager. À noter qu'il faudra prévoir un prétraitement de la question différent de celui qui est nécessaire pour la première composante.

In [20]:
with open("composante_conversationnelle/OpenSubtitles.et-fr.fr","r",encoding="utf-8" ) as fichier :
    corpus = fichier.read()#.replace("\n"," ")
    fichier.close()

In [21]:
len(corpus)

325689753

In [22]:
corpus=corpus[:640001]

Nous allons générer du texte via le lien au-dessus.

#### Apprentissage

In [23]:
import tensorflow as tf

import numpy as np
import os
import time
import datetime

In [24]:
# The unique characters in the file
vocab = sorted(set(corpus))
print ('{} unique characters'.format(len(vocab)))

118 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [25]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in corpus])

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [74]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(corpus)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

C
h
a
r
l


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [75]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

"Charles Chaplin in The Kid\npleurer.\nLa femme - dont le pêché était d'être mère.\nSeule.\nL'homme.\nSa pr"
"omenade matinale.\nEspèce d'empoté.\nvous avez perdu quelque chose.\nMerci d'aimer et de prendre soin de"
" cet orphelin\nC'est à vous ?\nComment s'appelle-t-il\nJohn\nCINQ ANS PLUS TARD\nMet la pièce dans le comp"
"teur à gaz.\nTu sais quelles rues nous allons faire aujourd'hui ?\nTout se passe bien...\nTravail numéro"
' 13.\nEn congé.\nLa femme - à présent une star célèbre.\nFélicitations pour votre interprétation de la n'


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [76]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [77]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "Charles Chaplin in The Kid\npleurer.\nLa femme - dont le pêché était d'être mère.\nSeule.\nL'homme.\nSa p"
Target data: "harles Chaplin in The Kid\npleurer.\nLa femme - dont le pêché était d'être mère.\nSeule.\nL'homme.\nSa pr"


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [78]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 31 ('C')
  expected output: 65 ('h')
Step    1
  input: 65 ('h')
  expected output: 58 ('a')
Step    2
  input: 58 ('a')
  expected output: 75 ('r')
Step    3
  input: 75 ('r')
  expected output: 69 ('l')
Step    4
  input: 69 ('l')
  expected output: 62 ('e')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [79]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [80]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [82]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [88]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 118)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.7728753


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [89]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [9]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [91]:
EPOCHS=10

In [92]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 99 steps
Epoch 1/10
99/99 [==============================] - 355s 4s/step - loss: 3.0115
Epoch 2/10
99/99 [==============================] - 319s 3s/step - loss: 2.2063
Epoch 3/10
99/99 [==============================] - 444s 4s/step - loss: 1.9966
Epoch 4/10
99/99 [==============================] - 496s 5s/step - loss: 1.8052
Epoch 5/10
99/99 [==============================] - 487s 5s/step - loss: 1.6499
Epoch 6/10
99/99 [==============================] - 490s 5s/step - loss: 1.5320
Epoch 7/10
99/99 [==============================] - 488s 5s/step - loss: 1.4431
Epoch 8/10
99/99 [==============================] - 488s 5s/step - loss: 1.3729
Epoch 9/10
99/99 [==============================] - 486s 5s/step - loss: 1.3159
Epoch 10/10
99/99 [==============================] - 486s 5s/step - loss: 1.2661


## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [10]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [13]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

NameError: name 'vocab_size' is not defined

In [107]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (1, None, 256)            30208     
_________________________________________________________________
gru_6 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_6 (Dense)              (1, None, 118)            120950    
Total params: 4,089,462
Trainable params: 4,089,462
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [138]:
model.save('my_model.h5') 

In [17]:
import tensorflow as tf
new_model = tf.keras.models.load_model('my_model.h5')

In [18]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate;
  num_generate = round((150-50)*np.random.random_sample() + 50)

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (''.join(text_generated))

In [26]:
print(generate_text(new_model, start_string="bonjour"))

s, que le temps ici, j'ai m'en fait rien a été ta fia


The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

### <font color="red">3. Mise en commun</font>

Vous avez maintenant tous les blocs nécessaires pour mettre en place le chatbot. Il suffit de les enchaîner :

+ lecture et prétraitement 1 de la question de l'usager ;
+ si pertinent, classification(s) de la question ; calcul(s) de similarité ;
+ retour d'une réponse extraite de la base ou prétraitement 2 et génération d'une réponse originale.

In [155]:
quest = str(input("Bonjour\n"))

while(quest != "0"):
    tableau = score_texte(quest)
    warnings.filterwarnings('ignore')
    print(max(tableau.iloc[:,1]))
    if max(tableau.iloc[:,1].mean(axis=1)) > 0.67 :
        res = data.loc[tableau.loc[tableau.iloc[:,1] == max(tableau.iloc[:,1])].index.item(),"Reponse"]
    else :
        res = generate_text(model, start_string=quest)
    print(res)
    quest = str(input("\n(Taper 0 pour quitter)"))

Bonjour
 dede


0.47
r votre preuve, d'ann.
Baisse-t-il ?
Il nous a dit...
Je vous suivais.
Arriède)
Au nomme Henvrey.
Ployage.
Vou



(Taper 0 pour quitter) hbjhb


0.59
le piécoupeau.
Sans tous !
Ce tont fondrait se reposer les falbrêtués.
Va feux, je viendrai.
Allez.
Allez



(Taper 0 pour quitter) 0


### <font color="red">4. Optionnel et expérimental : déploiement de l'application dans une interface graphique</font>

Cette étape est tout à fait optionnelle. N'y investissez pas votre temps au dépens des fonctionnalités de base de l'application.

Si vous en avez le temps et l'envie, tentez de déployer votre chatbot sur Facebook Messenger. Un exemple de solution est présenté [ici](https://www.datacamp.com/community/tutorials/facebook-chatbot-python-deploy). À noter que cette solution ne permet de garder le chatbot actif que tant que le code est exécuté en local. Pour que l'agent reste "éveillé" en permanence il faudrait exécuter le code sur un serveur externe qui resterait actif.

Après avoir passé énormément de temps pour mettre en place l'interface graphique via Facebook Messenger et n'ayant aucun résultat. Nous avons préféré faire une fenêtre graphique à l'aide de Tkinter

In [30]:
from tkinter import *

window = Tk()

input_user = StringVar()
input_field = Entry(window, text=input_user,background="cornflower blue")
input_field.pack(side=BOTTOM, fill=X)

def enter_pressed(event):
    input_get = input_field.get()
    quest = input_get
    tableau = score_texte(quest)
    
    if max(tableau.iloc[:,1]) > 0.67 :
        res = data.loc[tableau.loc[tableau.iloc[:,1] == max(tableau.iloc[:,1])].index.item(),"Reponse"]
        new_res=""
        for i in range(0,len(res)): #permet de rajouter un saut de ligne tous les 60 caractères afin d'afficher proprement la réponse
            if i % 60 ==0:
                new_res = new_res + "\n" + res[i]
            else:
                new_res = new_res + res[i]
    else :
        new_res = generate_text(new_model, start_string=quest)
        
    label = Label(frame, text=input_get,background="cornflower blue", anchor='e',width=200)
    champ_label = Label(frame, text=new_res,background="tan1", anchor='w',width=200)
    input_user.set('')
    label.pack()
    champ_label.pack()
    return "break"


frame = Frame(window, width=500, height=500)
frame.pack_propagate(False) # prevent frame to resize to the labels size
input_field.bind("<Return>", enter_pressed)
frame.pack()

window.mainloop()

### Évaluation du système

Le système consistant en plusieurs composantes de natures différentes, il est difficile de réaliser une évaluation quantitative automatique globale sur un jeu de test avec une vérité terrain. Il faudra évaluer chaque composante individuellement.

En particulier, pour la partie orientée tâche :
+ évaluation quantitative : créer un jeu de test (couples question-bonne réponse) et rapporter des mesures d'évaluation pertinentes : précision, rappel, F1 ;
+ évaluation qualitative : analyse manuelle d'un échantillon d'erreurs sur le jeu de test : nature de l'erreur, quelle semble en être la cause, etc.

La partie générative pourra être évaluée (avec l'ensemble du chatbot) par des utilisateurs, si vous êtes en mesure de constituer un petit échantillon de bénévoles désireux de tester votre bot. Dans ce cas demandez à vos utilisateurs, après avoir testé l'application, de répondre à un petit questionnaire avec quelques questions à échelle qui vous semblent pertinentes. Vous pouvez vous inspirer de la section 6 de [cet article](https://hal.archives-ouvertes.fr/hal-01309202/document) (évidemment, votre évaluation sera plus modeste). Cette évaluation est souhaitable, mais pas requise (s'il vous est impossible d'intéresser quelques utilisateurs). Il sera plus facile de faire tester l'application à des utilisateurs si elle est déployée dans une interface graphique.

### Rédaction d'un rapport

Le rapport doit aborder les points suivants :
+ Indiquez le lien vers un répertoire GitHub contenant vos différents fichiers `.py` et `pickle` et vos données.
+ Si pertinent, indiquez le lien vers votre chatbot sur Facebook Messenger.
+ Décrivez :
  + l'architecture et le fonctionnement de votre bot ;
  + la mise en oeuvre : votre approche pour les différentes composantes, vos choix et leur justification ;
  + la procédure d'évaluation et les résultats des évaluations que vous avez menées sur les différentes composantes.
+ Donnez votre ressenti personnel : comment trouvez-vous la performance du bot, quelles limites et points forts y voyez-vous, quelles difficultés avez-vous rencontrées dans la mise en oeuvre, qu'est-ce que vous aimeriez améliorer, etc.

## Suggestions

+ Vous êtes libres d'utiliser les approches que vous souhaitez : symboliques (règles, p. ex. expressions régulières), statistiques (apprentissage supervisé classique ou deep learning si vous avez suffisamment de données), un mélange des deux...
+ Vous pouvez également utiliser toutes les librairies et les modèles pré-entraînés qui vous semblent intéressants (par exemple les modèles statistiques de `spacy`, d'autres vecteurs pré-appris, etc.).
+ N'hésitez pas à être créatifs si vous avez d'autres idées de fonctionnalités que vous aimeriez intégrer.

## Critères d'évaluation du projet

+ Le bot fonctionne sans planter et fournit une réponse à toute sollicitation de l'usager.
+ Les réponses du bot sont globalement acceptables. On ne s'attendra pas à une très bonne performance, vu le peu de données disponibles pour la partie orientée tâche.
+ Toutes les fonctionnalités demandées ont été mises en oeuvre.
+ Le code est bien modularisé (fonctions, classes, modules selon les besoins).
+ Le code est correct du point de vue sémantique et syntaxique.
+ Le code est propre, lisible et bien annoté là où c'est nécessaire.
+ Une évaluation quantitative et qualitative a été réalisée.
+ Le rapport aborde tous les points requis avec suffisamment de détail.
+ Si pertinent : les sources sont citées (code, billets de blog, autres publications).
+ Bonus : des fonctionnalités supplémentaires ont été mises en oeuvre.
+ (Bonus : le chatbot est déployé dans une interface utilisateur. Impossible à évaluer si pas actif en permanence.)
+ Bonus : les TD ont bien été soumis au cours du semestre.

## Aspects pratiques

### Travail en groupes

+ Min 2, max 4 personnes par groupe
+ Composition des groupes à renseigner [ici](https://docs.google.com/spreadsheets/d/1TJOOG5zNrJ-7UZKVNGfV7rb704pDxSlEG2hTFGb5VM8/edit?usp=sharing) avant le lundi 2 décembre (il est possible de changer de groupe par la suite pour des raisons justifiées, sans pour autant en abuser)

### Fichiers à soumettre

Rapport au format `pdf`

### Modalité

Sur Cursus, dans l'espace du cours Fouille de textes, lien "Soumission du projet"

### Date limite

Vendredi 31 janvier 2020, 19h

## Pour aller plus loin...

Des approches, idées et outils à explorer pour la suite si vous êtes intéressés par le sujet :
+ Frameworks et librairies pour agents de dialogue :
    + https://rasa.com/docs/, https://github.com/RasaHQ/rasa
    + https://dialogflow.com/ (Google, closed source, API)
    + https://parl.ai/, https://github.com/facebookresearch/ParlAI (Facebook (FAIR), open source)
    + https://labs.cognitive.microsoft.com/en-us/project-conversation-learner, https://github.com/Microsoft/ConversationLearner-samples (Microsoft)
    + https://chatterbot.readthedocs.io/en/stable/, https://github.com/gunthercox/ChatterBot

+ Tutoriels :
    + http://www.ruiyan.me/pubs/tutorial-emnlp18.pdf
    + https://icml.cc/media/Slides/icml/2019/grandball(10-09-15)-10-13-00-4342-neural_approach.pdf (et en version rédigée : https://arxiv.org/abs/1809.08267)

+ Jeux de données pour les agents conversationnels :
    + https://github.com/sebastianruder/NLP-progress/blob/master/english/dialogue.md
    + https://breakend.github.io/DialogDatasets/
    + https://parl.ai/docs/tasks.html
    + https://arxiv.org/abs/1904.06472, https://github.com/PolyAI-LDN/conversational-datasets
    + https://www.microsoft.com/en-us/research/event/dialog-state-tracking-challenge/
    + https://github.com/google-research-datasets/dstc8-schema-guided-dialogue